In [ ]:
# %% [markdown]
# # NHG Vehicle Routing Case - Exploratory Data Analysis (EDA)
#
# **目标**:
# 1. 加载并清洗竞赛数据 (Orders, Locations, Distances)。
# 2. 验证案例描述中的关键统计指标 (如每日货量、门店分布)。
# 3. 可视化分析配送瓶颈 (高货量门店、地理分布、卸货时间)。

# %% [code]
# 1. 导入库与环境设置
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sys

# 设置绘图风格
sns.set(style="whitegrid")
# 解决 Matplotlib 中文显示问题 (如果需要)
plt.rcParams['axes.unicode_minus'] = False

# 自动定位项目根目录 (假设当前 notebook 在 notebooks/ 文件夹下)
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
data_dir = os.path.join(project_root, 'data')

print(f"工作目录: {current_dir}")
print(f"数据目录: {data_dir}")

# %% [code]
# 2. 加载与清洗数据
def load_and_clean_data(data_path):
    print("正在加载数据...")

    # --- A. 加载订单表 (Orders) ---
    orders_path = os.path.join(data_path, 'orders.csv')
    if not os.path.exists(orders_path):
        raise FileNotFoundError(f"找不到文件: {orders_path}")

    df_orders = pd.read_csv(orders_path)
    # 修复列名空格问题 (' CUBE ' -> 'CUBE')
    df_orders.columns = df_orders.columns.str.strip()

    # 过滤掉 ORDERID 为 0 的仓库点，只保留客户订单
    # 兼容可能的大小写问题
    if 'ORDERID' not in df_orders.columns:
        # 尝试找类似 'Order ID' 的列
        df_orders.rename(columns=lambda x: x.upper().replace(' ', ''), inplace=True)

    df_orders = df_orders[df_orders['ORDERID'] != 0].copy()

    # 类型转换
    df_orders['CUBE'] = pd.to_numeric(df_orders['CUBE'], errors='coerce')
    df_orders['TOZIP'] = pd.to_numeric(df_orders['TOZIP'], errors='coerce')

    # --- B. 加载位置表 (Locations) ---
    locs_path = os.path.join(data_path, 'locations.csv')
    df_locs = pd.read_csv(locs_path)
    df_locs.columns = df_locs.columns.str.strip()
    df_locs = df_locs.dropna(subset=['ZIP'])
    df_locs['ZIP'] = df_locs['ZIP'].astype(int)

    # --- C. 加载距离矩阵 (Distances) ---
    dist_path = os.path.join(data_path, 'distances.csv')
    df_dist = pd.read_csv(dist_path, index_col=0)
    # 尝试清理列名和索引为整数
    try:
        df_dist.columns = df_dist.columns.astype(float).astype(int)
        df_dist.index = df_dist.index.astype(float).astype(int)
    except:
        pass

    return df_orders, df_locs, df_dist

# 执行加载
df_orders, df_locs, df_dist = load_and_clean_data(data_dir)

# 合并经纬度信息以便画图
df_merged = df_orders.merge(df_locs[['ZIP', 'X', 'Y', 'CITY', 'STATE']],
                            left_on='TOZIP', right_on='ZIP', how='left')

print(f"✅ 数据加载完成！总订单数: {len(df_merged)}")
print(df_merged.head())

# %% [code]
# 3. 每日配送压力分析 (验证 Case Table 2)
# 统计每日的总货量、订单数、以及理论需要的最小车辆数
daily_stats = df_merged.groupby('DayOfWeek').agg(
    Total_Volume=('CUBE', 'sum'),
    Total_Stops=('ORDERID', 'count')
).reset_index()

# 按周一到周五排序
day_order = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
daily_stats['DayOfWeek'] = pd.Categorical(daily_stats['DayOfWeek'], categories=day_order, ordered=True)
daily_stats = daily_stats.sort_values('DayOfWeek')

# 计算理论最小卡车数 (容量 3200)
daily_stats['Min_Trailers'] = np.ceil(daily_stats['Total_Volume'] / 3200).astype(int)

print("--- 每日配送统计 (Table 2 验证) ---")
print(daily_stats)

# 可视化：每日货量对比
plt.figure(figsize=(10, 5))
sns.barplot(data=daily_stats, x='DayOfWeek', y='Total_Volume', palette='viridis', hue='DayOfWeek')
plt.axhline(y=3200, color='r', linestyle='--', label='单车容量上限 (3200 ft^3)')
plt.title('每日配送总货量 vs 单车容量')
plt.ylabel('体积 (CUBE)')
plt.legend()
plt.show()

# %% [code]
# 4. 识别“大客户” (高货量门店)
# 找出单次配送货量最大的门店，这些点通常是路径规划的难点
store_stats = df_merged.groupby(['TOZIP', 'CITY', 'STATE']).agg(
    Weekly_Freq=('DayOfWeek', 'count'),
    Avg_Volume=('CUBE', 'mean'),
    Max_Volume=('CUBE', 'max')
).reset_index()

top_volume_stores = store_stats.sort_values('Max_Volume', ascending=False).head(10)

print("--- 单次配送货量最大的 10 家门店 ---")
print(top_volume_stores)

# %% [code]
# 5. 门店地理分布可视化 (Map)
plt.figure(figsize=(10, 8))

# 画出所有门店点
sns.scatterplot(data=df_locs, x='X', y='Y', size=30, alpha=0.5, color='blue', label='Store')

# 标记 Top 5 高货量门店
top_5_zips = top_volume_stores.head(5)['TOZIP'].tolist()
top_5_locs = df_locs[df_locs['ZIP'].isin(top_5_zips)]
plt.scatter(top_5_locs['X'], top_5_locs['Y'], color='red', s=100, marker='*', label='Top 5 Volume Stores')

# 标记 DC (Wilmington, Zip 1887)
dc_loc = df_locs[df_locs['ZIP'] == 1887]
if not dc_loc.empty:
    plt.scatter(dc_loc['X'], dc_loc['Y'], color='black', s=150, marker='^', label='DC (Wilmington)')
    plt.text(dc_loc['X'].values[0]+0.1, dc_loc['Y'].values[0], 'DC', fontsize=12, fontweight='bold')

plt.title('NHG 门店与配送中心地理分布图')
plt.xlabel('经度 (Longitude)')
plt.ylabel('纬度 (Latitude)')
plt.legend()
plt.axis('equal')
plt.show()

# %% [code]
# 6. 卸货时间分析 (Unload Time)
# 规则: max(30, 0.03 * Volume)
df_merged['Unload_Time'] = df_merged['CUBE'].apply(lambda x: max(30, 0.030 * x))

# 绘制分布直方图
plt.figure(figsize=(10, 5))
sns.histplot(df_merged['Unload_Time'], bins=20, kde=True, color='green')
plt.axvline(x=30, color='r', linestyle='--', label='最小卸货时间 (30min)')
plt.title('订单卸货时间分布')
plt.xlabel('卸货时间 (分钟)')
plt.ylabel('订单数量')
plt.legend()
plt.show()

# 统计受限比例
min_time_impact = len(df_merged[df_merged['Unload_Time'] == 30])
total = len(df_merged)
print(f"受 '最小30分钟' 规则影响的订单数: {min_time_impact} (占 {min_time_impact/total:.1%})")